In [1]:
#copy files from s3 ( to be done only once)
#!aws s3 cp s3://ncw210data/NewClusterFiles/ /home/ubuntu/data/models_data/ --recursive

In [9]:
!mkdir /home/ubuntu/data/models_data/chartdatafiles

In [76]:
import pandas as pd
filename = '/home/ubuntu/data/models_data/smalldatafiles/small_cluster_data_0_0.csv'
#df_data = pd.read_csv('/home/ubuntu/data/smalldataoutput/small_cluster_data_0_0.csv',nrows = 10)

df_data = pd.read_csv(filename,nrows = 10,usecols=['appl_doc_number','appl_date','pub_date','number_of_days','count','patent_number','patent_date','patent_doc_kind','pat_number_of_days','length_of_grant','references_count','number_of_claims'])
df_data.columns.values[4] = 'appl_change_count'
print(df_data.columns)

Index(['appl_doc_number', 'appl_date', 'pub_date', 'number_of_days',
       'appl_change_count', 'patent_number', 'patent_date', 'patent_doc_kind',
       'pat_number_of_days', 'length_of_grant', 'references_count',
       'number_of_claims'],
      dtype='object')


In [78]:
from datetime import datetime
def create_charts_data_for_small_files(filename,p,c):
    df_data = pd.read_csv(filename,usecols=['appl_doc_number','appl_date','pub_date','number_of_days','count','patent_number','patent_date','patent_doc_kind','pat_number_of_days','length_of_grant','references_count','number_of_claims'])
    #rename the count column to appl_change_count
    df_data.columns.values[4] = 'appl_change_count'
    df_data['pub_year'] = df_data['pub_date'].apply(lambda x: datetime.strptime(str(x),'%Y%m%d').year)
    df_data['patent_date'] = df_data['patent_date'].apply(lambda x: str(x).rstrip('.0') if x else x)
    df_data['grant_year'] = df_data['patent_date'].apply(lambda x: datetime.strptime(str(x),'%Y%m%d').year if x and str(x).lower() != 'nan' else '')
    df_data['appl_year'] = df_data['appl_date'].apply(lambda x: datetime.strptime(str(x),'%Y%m%d').year)
    #round the year to nearest half year
    df_data['years_to_publish'] = df_data['number_of_days'].apply(lambda x: round(x/365 * 2.0) / 2.0)
    df_data['number_of_claims'] = df_data['number_of_claims'].apply(lambda x: round(x / 10) * 10 if x and str(x).lower() != 'nan' else x)
    df_data['years_to_grant'] = df_data['pat_number_of_days'].apply(lambda x: round(x/365 * 2.0) / 2.0 if x and str(x).lower() != 'nan' else x)
    
    df_appl_change_count = df_data.groupby(['appl_change_count']).size().reset_index(name='count')
    json_appl_change_count = df_appl_change_count.to_json(orient='records')
    
    df_publ_years = df_data.groupby(['pub_year']).size().reset_index(name='count')
    json_publ_years = df_publ_years.to_json(orient='records')
    
    df_grant_years = df_data.groupby(['grant_year']).size().reset_index(name='count')
    json_grant_years = df_grant_years.to_json(orient='records')
    
    df_appl_years = df_data.groupby(['appl_year']).size().reset_index(name='count')
    json_appl_years = df_appl_years.to_json(orient='records')
    
    df_years = df_data.groupby(['years_to_publish']).size().reset_index(name='count')
    json_years = df_years.to_json(orient='records')
    
    df_years_grant = df_data.groupby(['years_to_grant']).size().reset_index(name='count')
    json_years_grant = df_years_grant.to_json(orient='records')
    
    df_kind = df_data.groupby(['patent_doc_kind']).size().reset_index(name='count')
    json_kind= df_kind.to_json(orient='records')
    
    df_no_of_claims = df_data.groupby(['number_of_claims']).size().reset_index(name='count')
    json_no_of_claims= df_no_of_claims.to_json(orient='records')
      
    outputfilename = '/home/ubuntu/data/models_data/chartdatafiles/chart_cluster_data_'+ str(p) + '_' +str(c) + '.csv' 
    with open(outputfilename, "w") as chart_file:
        chart_file.write(json_publ_years + '\n' + json_appl_years + '\n' + json_grant_years +  '\n' + 
                        json_years + '\n' +json_years_grant + '\n' + json_kind + '\n' + json_appl_change_count + '\n' + json_no_of_claims)
    

In [79]:
#create chart files for second level clustered files
for p in range(0,10):
    for c in range(0,10):
        filename = '/home/ubuntu/data/models_data/smalldatafiles/small_cluster_data_'+ str(p) + '_' +str(c) + '.csv' 
        create_charts_data_for_small_files(filename,p,c)

/home/ubuntu/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [84]:
#create chart file for all data
import pandas as pd
from datetime import datetime
df_all = pd.DataFrame()
for p in range(0,10):
    for c in range(0,10):
        filename = '/home/ubuntu/data/models_data/smalldatafiles/small_cluster_data_'+ str(p) + '_' +str(c) + '.csv' 
        df_data = pd.read_csv(filename,usecols=['appl_doc_number','appl_date','pub_date','number_of_days','count','patent_number','patent_date','patent_doc_kind','pat_number_of_days','length_of_grant','references_count','number_of_claims'])
        df_all = df_all.append(df_data, ignore_index=True)
        #create_charts_data_for_small_files(filename,p,c)
        
print(df_all.shape)        

/home/ubuntu/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(4317210, 12)


In [85]:
#rename the count column to appl_change_count
df_all.columns.values[4] = 'appl_change_count'
df_all['pub_year'] = df_all['pub_date'].apply(lambda x: datetime.strptime(str(x),'%Y%m%d').year)
df_all['patent_date'] = df_all['patent_date'].apply(lambda x: str(x).rstrip('.0') if x else x)
df_all['grant_year'] = df_all['patent_date'].apply(lambda x: datetime.strptime(str(x),'%Y%m%d').year if x and str(x).lower() != 'nan' else '')
df_all['appl_year'] = df_all['appl_date'].apply(lambda x: datetime.strptime(str(x),'%Y%m%d').year)
#round the year to nearest half year
df_all['years_to_publish'] = df_all['number_of_days'].apply(lambda x: round(x/365 * 2.0) / 2.0)
df_all['number_of_claims'] = df_all['number_of_claims'].apply(lambda x: round(x / 10) * 10 if x and str(x).lower() != 'nan' else x)
df_all['years_to_grant'] = df_all['pat_number_of_days'].apply(lambda x: round(x/365 * 2.0) / 2.0 if x and str(x).lower() != 'nan' else x)

df_appl_change_count = df_all.groupby(['appl_change_count']).size().reset_index(name='count')
json_appl_change_count = df_appl_change_count.to_json(orient='records')

df_publ_years = df_all.groupby(['pub_year']).size().reset_index(name='count')
json_publ_years = df_publ_years.to_json(orient='records')

df_grant_years = df_all.groupby(['grant_year']).size().reset_index(name='count')
json_grant_years = df_grant_years.to_json(orient='records')

df_appl_years = df_all.groupby(['appl_year']).size().reset_index(name='count')
json_appl_years = df_appl_years.to_json(orient='records')

df_years = df_all.groupby(['years_to_publish']).size().reset_index(name='count')
json_years = df_years.to_json(orient='records')

df_years_grant = df_all.groupby(['years_to_grant']).size().reset_index(name='count')
json_years_grant = df_years_grant.to_json(orient='records')

df_kind = df_all.groupby(['patent_doc_kind']).size().reset_index(name='count')
json_kind= df_kind.to_json(orient='records')

df_no_of_claims = df_all.groupby(['number_of_claims']).size().reset_index(name='count')
json_no_of_claims= df_no_of_claims.to_json(orient='records')

outputfilename = '/home/ubuntu/data/models_data/chartdatafiles/chart_cluster_data_all_years.csv' 
with open(outputfilename, "w") as chart_file:
    chart_file.write(json_publ_years + '\n' + json_appl_years + '\n' + json_grant_years +  '\n' + 
                    json_years + '\n' +json_years_grant + '\n' + json_kind + '\n' + json_appl_change_count + '\n' + json_no_of_claims)


In [86]:
!cat /home/ubuntu/data/models_data/chartdatafiles/chart_cluster_data_all_years.csv

[{"pub_year":2005,"count":289051},{"pub_year":2006,"count":282012},{"pub_year":2007,"count":299485},{"pub_year":2008,"count":319807},{"pub_year":2009,"count":327418},{"pub_year":2010,"count":332383},{"pub_year":2011,"count":320309},{"pub_year":2012,"count":330594},{"pub_year":2013,"count":345949},{"pub_year":2014,"count":378841},{"pub_year":2015,"count":380897},{"pub_year":2016,"count":379758},{"pub_year":2017,"count":330706}]
[{"appl_year":1909,"count":1},{"appl_year":1911,"count":1},{"appl_year":1913,"count":1},{"appl_year":1991,"count":1},{"appl_year":1994,"count":2},{"appl_year":1995,"count":5},{"appl_year":1996,"count":7},{"appl_year":1997,"count":5},{"appl_year":1998,"count":4},{"appl_year":1999,"count":17},{"appl_year":2000,"count":131},{"appl_year":2001,"count":2507},{"appl_year":2002,"count":14376},{"appl_year":2003,"count":88930},{"appl_year":2004,"count":227917},{"appl_year":2005,"count":301574},{"appl_year":2006,"count":321336},{"appl_year":2007,"count":326925},{"appl_year"

In [87]:
#Copy files to S3
#!aws s3 cp /home/ubuntu/data/models_data/chartdatafiles/ s3://ncw210data/NewClusterFiles/chartdatafiles/  --recursive --only-show-errors

In [88]:
#!aws s3 cp /home/ubuntu/data/models_data/chartdatafiles/ s3://ncw210data/FullTextDataClusters/  --recursive --only-show-errors

In [68]:
#For test only
# p = 9
# c = 3
# filename = '/home/ubuntu/data/models_data/smalldatafiles/small_cluster_data_'+ str(p) + '_' +str(c) + '.csv' 
# create_charts_data_for_small_files(filename,p,c)

In [70]:
#!cat /home/ubuntu/data/models_data/chartdatafiles/chart_cluster_data_9_3.csv